In [1]:
import hail as hl

In [2]:
import pandas as pd
import h5py
# import h5py_cache
import sys 
import numpy as np
sys.path.insert(0, '../code/')
from importlib import reload 
import my_hail_helper as myhelper

In [3]:
myhelper = reload(myhelper)

In [4]:
covar, trait = myhelper.read_and_split_phenotype_csv(
    csv_path = '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/query_phenotypes_cleaned_up.csv',  # '/Users/yanyul/Documents/repo/github/ptrs-ukb/output/query_phenotypes_cleaned_up.csv',
    pheno_names = 'ht,meaning',
    covar_names = 'pc1',
    indiv_id = 'eid',
    int_names = None,
    str_names = 'meaning,eid'
)

In [5]:
indiv_list = myhelper.read_indiv_list('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-1.txt')

In [6]:
trait_sub = myhelper.subset_by_col(trait, 'eid', indiv_list)

In [7]:
indiv_in_sub_table = np.array(trait_sub['eid'].to_list()).astype('S10')

In [8]:
HDF5_CACHE = int(60 * (1024 ** 2))
f = h5py.File(
    '/homes/yanyul/test_predixcan/output/dev_full_along_sample10000_predixcan_prediction.h5', 
    'r'
)

In [9]:
f.keys()

<KeysViewHDF5 ['genes', 'pred_expr', 'samples']>

In [10]:
indiv_in_sub_table

array([b'1005003', b'1013480', b'1014217', ..., b'6017287', b'6022075',
       b'6023864'], dtype='|S10')

In [11]:
subset_idx_for_indiv_in_sub_table = np.where(np.isin(f['samples'][:], indiv_in_sub_table))

In [12]:
subset_idx_for_indiv_in_sub_table

(array([     1,      2,    129, ..., 487178, 487283, 487306]),)

In [13]:
subset_idx_for_indiv_in_sub_table[0].shape

(5000,)

In [14]:
# f['pred_expr'][1:100].shape
# np.array([i for i in range(10)])[::5]
list(subset_idx_for_indiv_in_sub_table[0])

[1,
 2,
 129,
 344,
 353,
 455,
 475,
 840,
 923,
 1003,
 1012,
 1099,
 1302,
 1394,
 1646,
 1841,
 2155,
 2257,
 2285,
 2677,
 2911,
 3035,
 3136,
 3161,
 3167,
 3226,
 3232,
 3250,
 3302,
 3312,
 3386,
 3395,
 3442,
 3707,
 3747,
 4018,
 4180,
 4192,
 4260,
 4262,
 4351,
 4397,
 4570,
 4579,
 4752,
 4825,
 4838,
 4867,
 4944,
 4986,
 5055,
 5534,
 5543,
 5784,
 5813,
 5827,
 5903,
 5973,
 6316,
 6378,
 6423,
 6669,
 6685,
 7013,
 7393,
 7484,
 7499,
 7545,
 7640,
 7815,
 7846,
 7992,
 8011,
 8071,
 8073,
 8160,
 8208,
 8239,
 8309,
 8454,
 8585,
 8596,
 8686,
 8730,
 8738,
 8791,
 8909,
 8957,
 8958,
 9300,
 9376,
 9378,
 9825,
 9910,
 9933,
 10063,
 10286,
 10313,
 10495,
 10546,
 10606,
 10680,
 10734,
 10754,
 10795,
 11169,
 11265,
 11455,
 11503,
 11519,
 11524,
 11705,
 11932,
 12002,
 12021,
 12149,
 12237,
 12359,
 12382,
 12541,
 12597,
 12681,
 12948,
 12962,
 13002,
 13150,
 13172,
 13206,
 13247,
 13420,
 13486,
 13513,
 13659,
 13674,
 13823,
 14071,
 14114,
 14179,
 142

In [15]:
pred_mat_sub = f['pred_expr'][:, list(subset_idx_for_indiv_in_sub_table[0])]  #

In [16]:
samples_sub = f['samples'][list(subset_idx_for_indiv_in_sub_table[0])]

In [17]:
pred_mat_sub[pred_mat_sub.sum(axis = 1) != 0,:].shape

(241, 5000)

In [18]:
genes_sub = f['genes'][pred_mat_sub.sum(axis = 1) != 0]

In [19]:
pred_mat_sub.shape

(7044, 5000)

In [20]:
df = pd.DataFrame(pred_mat_sub[pred_mat_sub.sum(axis = 1) != 0,:])

In [21]:
df.columns = samples_sub.astype('str')
df.index = genes_sub.astype('str')

In [22]:
df

,5595764,5172041,1528789,1517276,2468738,4556757,1484605,4185318,2261357,1221294,...,1937089,4882011,3683366,5577564,1984815,2637074,3686944,2969425,2874543,5129226
ENSG00000025708.13,0.260963,0.063263,0.377763,0.190563,0.378063,0.327563,0.322063,0.185563,0.095363,0.258863,...,0.256456,0.353156,0.235856,0.179756,0.189556,0.350756,0.354856,0.182356,0.323356,0.351256
ENSG00000025770.18,-0.000180,0.000820,0.001020,0.000920,0.001020,0.000920,0.001020,0.000000,0.002020,0.000820,...,0.001020,0.001020,0.000820,0.001020,0.002120,0.002120,0.002120,-0.000180,0.001020,0.001020
ENSG00000054611.13,0.054456,0.043356,0.056656,0.066356,0.043156,0.031556,0.059956,0.043356,0.021256,-0.003144,...,0.006934,0.093234,0.046734,0.022734,0.000334,0.054534,0.004734,0.076634,0.063734,0.036234
ENSG00000069998.12,0.087278,0.032478,0.072378,0.109678,0.159778,0.106278,0.087378,0.047078,0.046278,0.045378,...,0.095196,0.027596,0.031296,0.140096,0.031096,0.089296,0.065596,0.068696,0.061396,0.122696
ENSG00000070371.15,-0.439909,-0.113709,-0.127509,-0.444709,-0.440809,-0.146409,-0.277809,0.091191,-0.326709,0.054791,...,-0.120991,-0.463891,-0.326691,-0.136791,-0.027491,-0.242191,-0.231191,0.082309,0.075609,-0.245691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000273366.1,0.000637,-0.077363,-0.119463,-0.051763,0.014037,0.011137,0.001037,0.069537,-0.097463,-0.057463,...,-0.127562,-0.092062,-0.002562,-0.012362,0.048738,-0.095962,-0.115862,0.064938,-0.022962,-0.046462
ENSG00000278195.1,-0.179076,-0.370376,0.001424,-0.207476,-0.336376,-0.016376,-0.124276,-0.056376,-0.365976,-0.344176,...,-0.207936,-0.201236,-0.194936,-0.350736,-0.368536,-0.125636,-0.195436,-0.028636,-0.365836,-0.119236
ENSG00000279278.1,0.323373,0.701373,0.446673,0.238173,-0.018127,0.369273,-0.047327,-0.005627,0.318373,0.317573,...,0.266273,-0.012827,0.395073,0.597273,0.644273,0.323073,0.045673,0.414573,0.449673,0.742373
ENSG00000281538.1,0.069265,0.114665,0.127565,-0.039135,-0.076435,-0.079635,-0.125335,-0.122135,0.116065,-0.044935,...,0.125594,0.135094,0.074094,-0.045906,-0.106406,0.130394,0.136594,0.053694,0.067194,0.131794


In [27]:
df_melt = pd.melt(df.reset_index(), id_vars = 'index').rename(columns = {'index' : 'gene', 'variable' : 'eid', 'value' : 'pred_expr'})

In [28]:
df_melt.shape

(1205000, 3)

In [29]:
test = hl.Table.from_pandas(df_melt)

In [34]:
mt_expr = test.to_matrix_table(row_key = ['eid'], col_key = ['gene'])

In [35]:
mt_expr.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'gene': str
----------------------------------------
Row fields:
    'eid': str
----------------------------------------
Entry fields:
    'pred_expr': float64
----------------------------------------
Column key: ['gene']
Row key: ['eid']
----------------------------------------
